In [1]:
%run PilotForestAreas.ipynb

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
import param as pm
import panel as pn
import pandas as pd
import datetime as dt
import numpy as np
pn.extension()

In [3]:
f1.sale_value()

2834750.1707499996

In [4]:
f1.forested_area_hectares

7478

In [5]:
f1.acres_protected()

18470.66

In [6]:
f1.tons_carbon_per_year

21867.58

In [23]:
class Carbon(pm.Parameterized):
    forest = pm.Selector(default=f1, objects=[f1,f2])
    initial_carbon_price = pm.Number(10, bounds=(0,None))
    gbr_multiple_carbon_price = pm.Number(2.9, bounds=(0,None))
    carbon_price_appreciation = pm.Number(0.15, bounds=(0,1), step=0.01)
    stewardship_budget_increase = pm.Number(0.075, bounds=(0,1), step=0.01)
    tax_rate = pm.Number(0.2, bounds=(0,1), step=0.01)
    centree_admin_commission = pm.Number(0.15, bounds=(0,1), step=0.01)
    pfa_commission = pm.Number(0.35, bounds=(0,1), step=0.01)
    land_title_holders_commission = pm.Number(0.50, bounds=(0,1), step=0.01)
    below_soil_tonnage_factor = pm.Number(1, bounds=(0,2), step=0.01)
    minimum_yield = pm.Number(1.03)
    
    def compute(self, periods = 10, freq = 'Y'):
        
        df = pd.DataFrame(index=pd.date_range(dt.datetime.now(), periods=periods, freq=freq))
        df['Average VCM Carbon Price/Tonne (growth estimate)'] = [self.initial_carbon_price * (1+self.carbon_price_appreciation)**x for x in range(periods)]
        
        df['GBR Carbon Price/Tonne (multiple of average VCM)'] = self.gbr_multiple_carbon_price * df['Average VCM Carbon Price/Tonne (growth estimate)']
        
        df['Carbon Price'] = df['GBR Carbon Price/Tonne (multiple of average VCM)']

        df['Gross Revenue'] = df['Carbon Price'] * self.forest.tons_carbon_per_year * (1 + self.below_soil_tonnage_factor)

        df['Net Revenue'] = df['Gross Revenue'] * (1 - self.tax_rate)

        df['Stewardship Budget'] = [self.forest.forested_area_hectares * self.forest.stewardship_rate_per_hectare * (1+self.stewardship_budget_increase)**x for x in range(periods)]

        df['Gross Profit'] = df['Net Revenue'] - df['Stewardship Budget']
        
        df['Cost of capital (minimum yield)'] = (self.minimum_yield-1) * self.forest.sale_value()
        
        df['Net Profit'] = df['Gross Profit'] - df['Cost of capital (minimum yield)']

        df['Centree (Admin/Commission)'] = df['Net Profit'] * self.centree_admin_commission

        df['PFA Holder Commission'] = df['Net Profit'] * self.pfa_commission
        
        df['Land Title Holders Commission'] = df['Net Profit'] * self.land_title_holders_commission
        
        df['FPT holder spread over Treasury yield (bps)'] = 10_000 * df['PFA Holder Commission'] / self.forest.sale_value()

        df['PFA Holder Return as Percentage of Initial Cost'] = (1-self.forest.insurance_pool_rate) * ((self.minimum_yield-1) + (df['PFA Holder Commission'] / self.forest.sale_value()))
        
        df.index.name = "Date"

        return df
    
    def view_model_table(self):
        return self.compute().reset_index().hvplot.table(height=600, width=1000)
    
    def view_model_normalized_chart(self):
        carbon = self.compute()
        carbon_normal = carbon / carbon.iloc[0] 
        carbon_normal = carbon_normal + np.random.randn(*carbon_normal.shape) / 10
        return pn.panel(carbon_normal.hvplot.line(title="Normalized Growth Projections of Carbon Revenue", height=340, width=1000))
    
    def view(self):
        return pn.Row(
            self, pn.Tabs(
                ("Carbon Model", self.view_model_table), 
                ("Normalized Projections", self.view_model_normalized_chart),
            ))

In [24]:
c1 = Carbon(forest=f1, below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)

In [25]:
c1.compute()

,Average VCM Carbon Price/Tonne (growth estimate),GBR Carbon Price/Tonne (multiple of average VCM),Carbon Price,Gross Revenue,Net Revenue,Stewardship Budget,Gross Profit,Cost of capital (minimum yield),Net Profit,Centree (Admin/Commission),PFA Holder Commission,Land Title Holders Commission,FPT holder spread over Treasury yield (bps),PFA Holder Return as Percentage of Initial Cost
Date,,,,,,,,,,,,,,
2023-12-31 22:05:56.974498,10.000000,29.000000,29.000000,6.341598e+05,5.073279e+05,209384.000000,2.979439e+05,85042.505123,2.129014e+05,31935.202632,74515.472807,106450.675439,262.864338,0.050658
2024-12-31 22:05:56.974498,11.500000,33.350000,33.350000,7.292838e+05,5.834270e+05,225087.800000,3.583392e+05,85042.505123,2.732967e+05,40994.509392,95653.855247,136648.364639,337.433105,0.057369
2025-12-31 22:05:56.974498,13.225000,38.352500,38.352500,8.386764e+05,6.709411e+05,241969.385000,4.289717e+05,85042.505123,3.439292e+05,51589.379916,120375.219803,171964.599719,424.641371,0.065218
2026-12-31 22:05:56.974498,15.208750,44.105375,44.105375,9.644778e+05,7.715823e+05,260117.088875,5.114652e+05,85042.505123,4.264227e+05,63963.398849,149247.930649,213211.329498,526.494123,0.074384
2027-12-31 22:05:56.974498,17.490062,50.721181,50.721181,1.109149e+06,8.873196e+05,279625.870541,6.076937e+05,85042.505123,5.226512e+05,78397.682292,182927.925348,261325.607640,645.305280,0.085077
2028-12-31 22:05:56.974498,20.113572,58.329358,58.329358,1.275522e+06,1.020418e+06,300597.810831,7.198197e+05,85042.505123,6.347772e+05,95216.582045,222172.024771,317388.606815,783.744638,0.097537
2029-12-31 22:05:56.974498,23.130608,67.078762,67.078762,1.466850e+06,1.173480e+06,323142.646644,8.503375e+05,85042.505123,7.652950e+05,114794.251088,267853.252540,382647.503628,944.891918,0.112040
2030-12-31 22:05:56.974498,26.600199,77.140577,77.140577,1.686878e+06,1.349502e+06,347378.345142,1.002124e+06,85042.505123,9.170813e+05,137562.199892,320978.466414,458540.666306,1132.298958,0.128907
2031-12-31 22:05:56.974498,30.590229,88.711663,88.711663,1.939909e+06,1.551928e+06,373431.721027,1.178496e+06,85042.505123,1.093453e+06,164017.992624,382708.649455,546726.642079,1350.061298,0.148506


In [26]:
c1.view()

Row
    [0] Column(margin=5, name='Carbon', width=300)
        [0] StaticText(value='<b>Carbon</b>')
        [1] Row(width_policy='max')
            [0] Select(margin=(5, 0, 5, 10), name='Forest', options=OrderedDict([('Forest00786...]), value=Forest, width=240)
            [1] Toggle(align='end', button_type='primary', height_policy='fit', margin=(0, 0, 5, 10), max_height=30, max_width=20, name='⋮')
        [2] FloatInput(name='Initial carbon price', start=0, value=10)
        [3] FloatInput(name='Gbr multiple c..., start=0, value=2.9)
        [4] FloatSlider(name='Carbon price a..., step=0.01, value=0.15)
        [5] FloatSlider(name='Stewardship b..., step=0.01, value=0.075)
        [6] FloatSlider(name='Tax rate', step=0.01, value=0.2)
        [7] FloatSlider(name='Centree admin c..., step=0.01, value=0.15)
        [8] FloatSlider(name='Pfa commission', step=0.01, value=0.35)
        [9] FloatSlider(name='Land title holders c..., step=0.01, value=0.5)
        [10] FloatSlider(end=2, name='Below soil tonnage f..., step=0.01)
        [11] FloatInput(name='Minimum yield', value=1.03)
    [1] Tabs
        [0] ParamMethod(method, _pane=HoloViews, name='Carbon Model')
        [1] ParamMethod(method, _pane=HoloViews, name='Normalized Projections')